In [1]:
import sys
import os
from attrs import define, frozen, field
from functools import cached_property
from IPython.display import Image, SVG, display
current_dir = os.getcwd()
sys.path.append(current_dir[:-9])

In [2]:
from qrew.simulation.refactor.Process_Library.chemistry_vqe import *
from qrew.simulation.refactor.broker import Broker
from qrew.simulation.refactor.quantum import QuantumCircuit
from qrew.simulation.refactor.devices.quantum_devices import *
from qrew.simulation.refactor.data import Data
from openfermion.ops import FermionOperator, QubitOperator
from qrew.simulation.refactor.resources.classical_resources import (
    ClassicalDevice,
)

qiskit version: 1.2.4 - quantum_resources.py - get_transpiled_swap()
qiskit version: 1.2.4 - quantum_resources.py - get_transpiled_swap()
qiskit version: 1.2.4 - quantum_resources.py - get_transpiled_swap()


In [3]:
basis = "sto-3g"
symbols = ["H", "H", "H", "H"]
coordinates = [
    [0.7071, 0.0, 0.0],
    [0.0, 0.7071, 0.0],
    [-1.0071, 0.0, 0.0],
    [0.0, -1.0071, 0.0],
]
coordinates = [
    [elmt * ANGSTROM_TO_BOHR for elmt in coordinates[idx]]
    for idx in range(len(coordinates))
]
charge = 0
geometry_model = "RHF"
starting_inputs = [
    [
        Data(data=basis, properties={"Usage": "Basis"}),
        Data(data=symbols, properties={"Usage": "Atomic Symbols"}),
        Data(data=coordinates, properties={"Usage": "Coordinates"}),
        Data(data=charge, properties={"Usage": "Total Charge"}),
        Data(data=geometry_model, properties={"Usage": "Geometry Model"}),
    ]
]



In [4]:

def gen_broker():
    supercomputer = ClassicalDevice(
        device_name="Supercomputer",
        processor_type="CPU",
        RAM=100 * 10**9,
        properties={"Cores": 20, "Clock Speed": 3 * 10**9},
    )

    broker = Broker(
        classical_devices=[supercomputer],
        quantum_devices=[
            IBM_Brisbane,
         
        
            IBM_Kyiv,
      
            IBM_Sherbrooke,
        ],
    )
    return broker


def test_construct_ref_circuit_network_JW():
    init_mol = Node(process_model=GSE_InitMolecule, network_type="INPUT")
    nuc_repl = Node(process_model=GSE_ComputeNuclearRepulsion, network_type="NETWORK")
    init_basis_set = Node(
        process_model=GSE_InitBasisSet, network_type="NETWORK", extend_dynamic=True
    )
    core = Node(process_model=GSE_ComputeCore, network_type="NETWORK")
    repulsion = Node(process_model=GSE_ComputeRepulsion, network_type="NETWORK")
    overlap = Node(process_model=GSE_ComputeOverlap, network_type="NETWORK")
    overlap_eigs = Node(
        process_model=GSE_ComputeEigs, network_type="NETWORK", matrix_name="Overlap"
    )
    trnsfrm_mat = Node(process_model=GSE_ComputeTrnsfrmMat, network_type="NETWORK")
    scf_itr = Node(
        process_model=GSE_SCFItr, network_type="NETWORK", extend_dynamic=True
    )
    init_ansatz = Node(
        process_model=GSE_InitAnsatz, network_type="INPUT", extend_dynamic=True
    )
    ref_circuit = Node(process_model=GSE_GenRefCircuit, network_type="OUTPUT")

    init_mol.insert_output_node(init_basis_set)
    init_mol.insert_output_node(core)
    init_mol.insert_output_node(nuc_repl)
    init_basis_set.insert_output_node(core)
    init_basis_set.insert_output_node(overlap)
    init_basis_set.insert_output_node(repulsion)
    overlap.insert_output_node(overlap_eigs)
    overlap_eigs.insert_output_node(trnsfrm_mat)

    init_mol.insert_output_node(scf_itr)
    nuc_repl.insert_output_node(scf_itr)
    core.insert_output_node(scf_itr)
    repulsion.insert_output_node(scf_itr)
    trnsfrm_mat.insert_output_node(scf_itr)

    init_mol.insert_output_node(init_ansatz)
    scf_itr.insert_output_node(init_ansatz)

    init_ansatz.insert_output_node(ref_circuit)

    network = Network(
        nodes=[
            init_mol,
            init_basis_set,
            core,
            repulsion,
            overlap,
            overlap_eigs,
            trnsfrm_mat,
            scf_itr,
            init_ansatz,
            ref_circuit,
        ],
        input_nodes=[init_mol],
        output_nodes=[ref_circuit],
        broker=gen_broker(),
    )

    add_inputs = [
        [
            Data(data="JW", properties={"Usage": "Qubit Mapping"}),
            Data(data="UCCSD", properties={"Usage": "Ansatz"}),
            Data(data="random", properties={"Usage": "Ansatz Params"}),
        ]
    ]
    df = network.run(
        starting_nodes=[init_mol, init_ansatz],
        starting_inputs=starting_inputs + add_inputs,
    )

    output_edges = network.output_nodes[0].output_edges
    assert len(output_edges) == 1

    assert output_edges[0].data[0].properties["Data Type"] == QuantumCircuit
    assert output_edges[0].data[0].properties["Usage"] == "Reference Circuit"

    ref_circuit = output_edges[0].data[0].data
    assert len(ref_circuit.gate_set) == 1

test_construct_ref_circuit_network_JW()


Pre-flight: GSE_InitMolecule.N1 READY - graph.py - run()
Skipping pre-flight for dynamic node GSE_InitAnsatz.N10 - graph.py - run()
Starting node GSE_InitMolecule.N1 with inputs: [Data(`d1`, usage: Basis), Data(`d2`, usage: Atomic Symbols), Data(`d3`, usage: Coordinates), Data(`d4`, usage: Total Charge), Data(`d5`, usage: Geometry Model)] ---> resources allocated: Allocation(1, RAM= 4.2e-05MB) - graph.py - __initialize_node()
Init-skip GSE_InitAnsatz.N10: Input data properties validation failed:
- Data id=d8, object=Data(`d8`, usage: Ansatz Params)
    • Usage: expected=Geometry Model actual=Ansatz Params - graph.py - run()
Init-skip GSE_InitAnsatz.N10: Input data properties validation failed:
- Data id=d8, object=Data(`d8`, usage: Ansatz Params)
    • Usage: expected=Molecular Orbital Occupancies actual=Ansatz Params
    • Data Type: expected=<class 'list'> actual=<class 'str'> - graph.py - run()
Starting node GSE_ComputeNuclearRepulsion.N2 with inputs: [Data(`d13`, usage: Coordinates

/Users/sophieblock/miniforge3/envs/torch_env/lib/python3.11/site-packages/autoray/autoray.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
